# Deep Learning - Nasir Hussain - 2021/10/02

# 12 Generative deep learning

- artificial intelligence to emulate
  - passive tasks
    - object recognition
  - reactive tasks
    - driving a car
  - creative activities
- more augmented intelligence than artificial intelligence
- A large part of artistic creation consists of simple pattern recognition and technical skill
- Our perceptual modalities, our language, and our artwork all have statistical structure. Learning this structure is what deep learning algorithms excel at.
-  in the hands of a skilled artist, algorithmic generation can be steered to become meaningful and beautiful
- Topics
  - sequence data generation
  - DeepDream
  - image generation
    - variational autoencoders
    - generative adversarial networks.

## 12.1 Text generation


- recurrent neural networks used to
  - generate sequence data for
    - text generation
    - sequences of musical notes to generate new music
    - brushstroke data generate paintings
- Sequence data generation used for
  - artistic content generation
  - speech synthesis
  - dialogue generation for chatbots

### 12.1.1 A brief history of generative deep learning for sequence generation

- LSTM
  - used early on to generate text character by character
  - music generation
- recurrent networks
  - sequence data generation
  - text and dialogue generation
  - music generation
  - speech synthesis.
- recurrent mixture density networks 
  - to generate human-like handwriting 
- Transformer architecture
  - supervised natural language processing
  - generative sequence models
  - language modeling (word-level text generation)
    - GPT-3

### 12.1.2 How do you generate sequence data?

- The universal way to generate sequence data in deep learning is to train a model to predict the next token or next few tokens in a sequence, using the previous tokens as input.
- language model
  - any network that can model the probability of the next token given the previous ones
- trained language model (character-level neural language model)
  - sample from it (generate new sequences)
  - initial string of text (called conditioning data)
  -  add the generated output back to the input data
  - repeat the process many times

### 12.1.3 The importance of the sampling strategy

- way you choose the next token is crucially important
  - greedy sampling
    - consisting of always choosing the most likely next character.
    - repetitive, predictable strings
    -  sampling from a probability distribution
      - one where a certain word has probability 1 and all others have probability 0.
  - stochastic sampling
    - it introduces randomness in the sampling process by sampling from the probability distribution for the next character 
- softmax output of the model is neat
  - it allows even unlikely words to be sampled some of the time, generating more interesting-looking sentences and sometimes showing creativity by coming up with new, realistic-sounding sentences that didn’t occur in the training data
  - it doesn’t offer a way to control the amount of randomness in the sampling process

- Why would you want more or less randomness?
- pure random sampling
  - maximum randomness
  - higher entropy
  - more surprising and creative sequences
- greedy sampling
  - No randomness
  - lower entropy
  - more predictable structure
- softmax function
  - constitutes an intermediate point between these two extremes
- softmax temperature
  - control the amount of stochasticity
  - characterizes the entropy of the probability distribution used for sampling
  - Higher temperatures result in sampling distributions of higher entropy
  - lower temperature will result in less randomness

In [ ]:
# Listing 12.1 Reweighting a probability distribution to a different temperature
import numpy as np 
def reweight_distribution(original_distribution, temperature=0.5):
  distribution = np.log(original_distribution) / temperature
  distribution = np.exp(distribution)
  return distribution / np.sum(distribution)

### 12.1.4 Implementing text generation with Keras (IMDB movie review dataset)

- our language model will be a model of the style and topics of these movie reviews specifically, rather than a general model of the English language.


#### PREPARING THE DATA

In [ ]:
# Listing 12.2 Downloading and uncompressing the IMDB movie reviews dataset
!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

--2022-06-03 20:18:14--  https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  33.5MB/s    in 2.4s    

2022-06-03 20:18:17 (33.5 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [ ]:
# Listing 12.3 Creating a dataset from text files (one file = one sample)
import tensorflow as tf 
from tensorflow import keras
dataset = keras.utils.text_dataset_from_directory(
    directory="aclImdb", label_mode=None, batch_size=256)
dataset = dataset.map(lambda x: tf.strings.regex_replace(x, "<br />", " "))

Found 100006 files belonging to 1 classes.


In [ ]:
# Listing 12.4 Preparing a TextVectorization layer
from tensorflow.keras.layers import TextVectorization
 
# only use first 100 words from the sequence
sequence_length = 100
# max words with higest occurance rest will be considered as "UNK"
vocab_size = 15000
text_vectorization = TextVectorization(
    max_tokens=vocab_size, 
    output_mode="int",
    output_sequence_length=sequence_length,
)
text_vectorization.adapt(dataset)

In [ ]:
# Listing 12.5 Setting up a language modeling dataset
# input samples are vectorized texts, 
# and corresponding targets are the same texts offset by one word.


def prepare_lm_dataset(text_batch):
  vectorized_sequences = text_vectorization(text_batch)
  x = vectorized_sequences[:, :-1]
  y = vectorized_sequences[:, 1:]
  return x, y
 
lm_dataset = dataset.map(prepare_lm_dataset, num_parallel_calls=1)

#### A TRANSFORMER-BASED SEQUENCE-TO-SEQUENCE MODEL

- We’ll train a model to predict a probability distribution over the next word in a sentence, given a number of initial words. 
- When the model is trained, we’ll feed it with a prompt, sample the next word, add that word back to the prompt, and repeat, until we’ve generated a short paragraph.


- we could train a model that takes as input a sequence of N words and simply predicts word N+1
  - N words are needed to be given as prompt but it is not usefull as we have 100 words in a sample
  - Many of the training sequenece will be overlapping so lot of redundant work will be carried out by the model.

- To address these two issues we’ll use a sequence-to-sequence model
  - we’ll feed sequences of N words into our model, and we’ll predict the sequence offset by one
  - We’ll use causal masking to make sure that, for any i , the model will only be using words from 0 to i in order to predict the word i + 1. 
  - This means that we’re simultaneously training the model to solve N mostly overlapping but different problems: 
  - predicting the next words given a sequence of 1 <= i <= N prior words
  - At generation time, even if you only prompt the model with a single word, it will be able to give you a probability distribution for the next possible words.


- setup you can use for sequence-to-sequence learning in the general case: 
  - feed the source sequence into an encoder, and then feed both the encoded sequence and the target sequence into a decoder that tries to predict the same target sequence offset by one step. 
- When you’re doing text generation, there is no source sequence: 
  - you’re just trying to predict the next tokens in the target sequence given past tokens, which we can do using only the decoder. And thanks to causal padding, the decoder will only look at words 0...N to predict the word N+1

In [ ]:
# PositionalEmbedding and TransformerDecoder
import tensorflow as tf
from tensorflow.keras import layers

class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
          num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
          num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super(TransformerDecoder, self).get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

In [ ]:
# Listing 12.6 A simple Transformer-based language model
from tensorflow.keras import layers
embed_dim = 256
latent_dim = 2048
num_heads = 2
 
inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, x)
outputs = layers.Dense(vocab_size, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop")

### 12.1.5 A text-generation callback with variable-temperature sampling

In [ ]:
# Listing 12.7 The text-generation callback

import numpy as np
 
# Dict that maps word indices back to strings, to be used for text decoding
tokens_index = dict(enumerate(text_vectorization.get_vocabulary()))
 
#  Implements variable-temperature sampling from a probability distribution
def sample_next(predictions, temperature=1.0):
  predictions = np.asarray(predictions).astype("float64")
  predictions = np.log(predictions) / temperature
  exp_preds = np.exp(predictions)
  predictions = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, predictions, 1)
  return np.argmax(probas)

class TextGenerator(keras.callbacks.Callback):
  def __init__(
      self,
      prompt,
      generate_length,
      model_input_length,
      temperatures=(1.,),
      print_freq=1):
    self.prompt = prompt
    self.generate_length = generate_length
    self.model_input_length = model_input_length
    self.temperatures = temperatures
    self.print_freq = print_freq
  
  def on_epoch_end(self, epoch, logs=None):
    if (epoch + 1) % self.print_freq != 0:
      return
    for temperature in self.temperatures:
      print("== Generating with temperature", temperature)
      sentence = self.prompt
      for i in range(self.generate_length):
        tokenized_sentence = text_vectorization([sentence])
        predictions = self.model(tokenized_sentence)
        next_token = sample_next(predictions[0, i, :])
        sampled_token = tokens_index[next_token]
        sentence += " " + sampled_token
      print(sentence)

prompt = "This movie"
text_gen_callback = TextGenerator(
    prompt,
    generate_length=50,
    model_input_length=sequence_length,
    temperatures=(0.2, 0.5, 0.7, 1., 1.5))

In [ ]:
# Listing 12.8 Fitting the language model

# run on more epoch to get batter results say 200

model.fit(lm_dataset, epochs=10, callbacks=[text_gen_callback])

Epoch 1/10
391/391 [==============================] - ETA: 0s - loss: 5.0832== Generating with temperature 0.2
This movie movie was does very have difficult heard to three be points [UNK] characters which and was its written still and a hilarious camera nancy work drew was barrymore worth was watching presented as what a may bad have until been the screen two the its films awesome few the parts
== Generating with temperature 0.5
This movie movie gives is me not stunned too me much just watching dolls it academy unlike award most was of i the was best at romantic the comedy severely at shields imdb a outside time of let the them person if could it not with be such scary an i offer
== Generating with temperature 0.7
This movie is was spoiler by all its kinds acting of a a holiday bullying and who the unknown overall people plot who plot went is wrong absolutely with location a and muddled terrible ripoff plot of holes tank that next deserves day to in the water animal battles pocket need

- low temperature value results in very boring and repetitive text and can sometimes cause the generation process to get stuck in a loop.
- higher temperatures, the generated text becomes more interesting, surprising, even creative.
- very high temperature, the local structure starts to break down, and the output looks largely random

- training a bigger model, longer, on more data, you can achieve generated samples that look far more coherent and realistic than this one

---